##Some preliminaries, you will need 
- package Match (Pkg.add("Match")) 
- PyCall
- a working Python, with scipy, numpy and qutip installed.

In [1]:
reload("PTSM.jl")
reload("Initial.jl")
reload("Symplectic.jl")

In [2]:
using PyCall

In [3]:
@pyimport scipy.optimize as so
@pyimport numpy as np
@pyimport matplotlib.pyplot as plt
@pyimport qutip as qt
@pyimport numpy.linalg as la

## We need to get the Python finite field package working

In [4]:
unshift!(PyVector(pyimport("sys")["path"]), "./finite-fields/")
@pyimport finitefield as ff

In [5]:
F22 = ff.FiniteField(2,2)

1 + 0 x^1 + 1 x^2
1 + 0 x^1 + 1 x^2
0 + 0 x^1 + 1 x^2
0 + 1 x^1 + 1 x^2
0 + 1 x^1 + 1 x^2
0 + 1 x^1 + 1 x^2
1 + 1 x^1 + 1 x^2


fn (generic function with 1 method)

In [6]:
@pyimport SLFunctions as sl

In [7]:
function getStabilisers()
    jstabs=Array{Int64,2}[]
    stabs = sl.getF2Stabilisers()
    for i in 1:60
        push!(jstabs,reshape(stabs[i,:,:],4,4))
    end
    return jstabs
end


getStabilisers (generic function with 1 method)

In [8]:
stabs=getStabilisers();

##So we have the F22 stabilisers (60 of them) loaded into stabs

#Optimise the gates required for these 60
##!WARNING
these take a long time to run! - The data is now saved, just load it, next worksheet.

In [9]:
#watch out currently fails for 29 (as its the identity)
#gatesNeeded=Array{Int32,1}[]
#for i=1:60
#    state=setup(2)
#    state[:,1:4]=stabs[i]
#    back=bruteForceBreadthFirst(state)
#    push!(gatesNeeded,back)
#    writedlm("gateList$i.csv",back)
#end
    

In [10]:
#since we saved them previously we can load them
gates=Array{Float64}[]
for i=1:60
    push!(gates,readdlm("compiledGates/gateList$i.csv"))
end

In [11]:
gates[29]=[7.0]

1-element Array{Float64,1}:
 7.0

##The saved gates are indexes to the following "gate" array

In [12]:
gatesCommand = (Expr)[]
	svec = []
	for i = 1:2
	  for j = 1:2
	  	if i!=j
            push!(gatesCommand,Expr(:call,:cnot,:svec,i,j,false))
	  	end
	  end
    push!(gatesCommand,Expr(:call,:hadamard,:svec,i,false))
    push!(gatesCommand,Expr(:call,:phase,:svec,i,false))
	end
push!(gatesCommand,Expr(:call,:noop))

7-element Array{Expr,1}:
 :(cnot(svec,1,2,false))  
 :(hadamard(svec,1,false))
 :(phase(svec,1,false))   
 :(cnot(svec,2,1,false))  
 :(hadamard(svec,2,false))
 :(phase(svec,2,false))   
 :(noop())                

## Or if we just want to print it

In [13]:
gatestring = ASCIIString[]
svec = []
for i = 1:2
  for j = 1:2
  	if i!=j
            push!(gatestring,"cnot($i,$j)")
  	end
  end
  push!(gatestring,"hadamard($i)")
  push!(gatestring,"phase($i)")
end
push!(gatestring,"Identity")

7-element Array{ASCIIString,1}:
 "cnot(1,2)"  
 "hadamard(1)"
 "phase(1)"   
 "cnot(2,1)"  
 "hadamard(2)"
 "phase(2)"   
 "Identity"   

In [14]:
function makeFromCommand(command)
    currentState = [1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
    si = [1 0;0 1]
    sphase1=kron([1 0;0 im],si)
    sphase2=kron(si,[1 0;0 im])
    shadmard=1/sqrt(2)*[1 1;1 -1]
    shad1 = kron(shadmard,si)
    shad2 = kron(si,shadmard)
    cnot12 = [1 0 0 0;0 1 0 0;0 0 0 1;0 0 1 0]
    cnot21 = [1 0 0 0;0 0 0 1;0 0 1 0;0 1 0 0]
    for t in command
        m = match(r"setup\((.*)\)",t)
        if (m!=nothing)
           state=[1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
        else 
           m=match(r"phase\((.*)\)",t)
           if (m!=nothing)
                bit = int(m.captures[1])
                if bit == 1
                   currentState=currentState*sphase1
                else 
                   currentState=currentState*sphase2
                end
            else
                m=match(r"hadamard\((.*)\)",t)
                if (m!=nothing)
                    bit = int(m.captures[1])
                    if bit == 1
                        currentState=currentState*shad1
                    else 
                        currentState=currentState*shad2
                    end
                else
                    m=match(r"cnot\((.*),(.*)\)",t)
                    if (m!=nothing)
                        cbit = int(m.captures[1])
                        tbit = int(m.captures[2])
                        if cbit ==1 
                            currentState = currentState*cnot12
                        else
                            currentState = currentState*cnot21
                        end
                    end
                end
            end
        end
    end
    return  currentState=currentState
end


makeFromCommand (generic function with 1 method)

In [15]:
# if you haven't already you may need to: Pkg.add("ProgressMeter")
# or just remove the ProgressMeter references, they aren't actually needed.

In [16]:
using ProgressMeter

###Looks good, so lets make them all

In [237]:
sl2Cliffords= Array{Complex{Float64},2}[]
sl2CliffordCommands = Array{ASCIIString,1}[]
n=60*4*4
p=Progress(n,1)
for i=1:60
    for j=1:4
        for z=1:4
            state=setup(2)
            state[:,1:4]=stabs[i]
            decomposeState(state,true)
            #Apply the PAULIS!
            if (j==2)
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            elseif (j==3)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
            elseif (j==4)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            end
            if (z==2)
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            elseif (z==3)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
            elseif (z==4)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            end
            push!(sl2CliffordCommands,commands)
            push!(sl2Cliffords,makeFromCommand(commands))
            next!(p)
        end
    end
end

In [238]:
size(sl2Cliffords)

(960,)

In [19]:
sl2CliffordCommands[1]

4-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "output(svec)"

#so the number of gates is the length of the commands minus 2

In [20]:
sl2CliffordsLength = [size(sl2CliffordCommands[i],1)-2 for i=1:size(sl2CliffordCommands,1)]
print("Size is $(size(sl2CliffordsLength,1)) and mean is $(mean(sl2CliffordsLength))")

Size is 960 and mean is 17.2

##Check if it is unitary 2

In [21]:
sum = 0
numb = size(sl2Cliffords,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(sl2Cliffords[i])'*sl2Cliffords[j]))^4
    end
end
print("$sum -> $(sum/numb^2))\n")

1.8431999999999998e6 -> 1.9999999999999998)


##And the magic number is indeed 2

## Now lets compare with the actual full Clifford Group.

In [22]:
getNumberOfCliffords(2)

11520

In [23]:
getNumberOfSymplecticCliffords(2) # this is the number mapped to integers

720

In [24]:
getNumberOfBitStringsCliffords(2) # this is the number we multiply it by

16

In [25]:
FullCliffords= Array{Complex{Float64},2}[]
FullCliffordCommands = Array{ASCIIString,1}[]
n=11520
p=Progress(n,1)
for i=0:719 # number of symplectics
    for j=0:15 # number of phase variations.
        state=setup(2)
        decompose(i,j,2,true,false) # symplectic i, bits j, 2 qubits, no output and rationalise
        push!(FullCliffordCommands,commands)
        push!(FullCliffords,makeFromCommand(commands))
        next!(p)
    end
end

Progress: 100% Time: 0:00:07


In [26]:
for i=0:15
    decompose(0,i,2,false,false)
end

Tableau for unitary: 
+XI
+IX
---
+ZI
+IZ
setup(2)
output(svec)
Tableau for unitary: 
-XI
+IX
---
+ZI
+IZ
setup(2)
phase(1)
phase(1)
output(svec)
Tableau for unitary: 
+XI
-IX
---
+ZI
+IZ
setup(2)
phase(2)
phase(2)
output(svec)
Tableau for unitary: 
-XI
-IX
---
+ZI
+IZ
setup(2)
phase(2)
phase(2)
phase(1)
phase(1)
output(svec)
Tableau for unitary: 
+XI
+IX
---
-ZI
+IZ
setup(2)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
-XI
+IX
---
-ZI
+IZ
setup(2)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
+XI
-IX
---
-ZI
+IZ
setup(2)
phase(2)
phase(2)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
-XI
-IX
---
-ZI
+IZ
setup(2)
phase(2)
phase(2)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
+XI
+IX
---
+ZI
-IZ
setup(2)
hadamard(2)
hadamard(1)
phase

In [27]:
for j=1:16
    print(j,"\n")
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [28]:
n=2
for bits=1:16
    top = [ (bits >> (x-1)) %2 for x=1:n]
    for co=1:n
        bits = div(bits,2)
    end
    bottom = [ (bits >> (x-1)) %2 for x=1:n]
    print(bits,"--->",top," ",bottom,"\n")
end

0--->[1,0] [0,0]
0--->[0,1] [0,0]
0--->[1,1] [0,0]
1--->[0,0] [1,0]
1--->[1,0] [1,0]
1--->[0,1] [1,0]
1--->[1,1] [1,0]
2--->[0,0] [0,1]
2--->[1,0] [0,1]
2--->[0,1] [0,1]
2--->[1,1] [0,1]
3--->[0,0] [1,1]
3--->[1,0] [1,1]
3--->[0,1] [1,1]
3--->[1,1] [1,1]
4--->[0,0] [0,0]


In [29]:
n=2
bits=1
top = [ (bits >> (x-1)) %2 for x=1:n]
bits = div(bits,2)
bottom = [ (bits >> (x-1)) %2 for x=1:n]
print(top," ",bottom)

{1,0} {0,0}

In [30]:
top

2-element Array{Any,1}:
 1
 0

In [31]:
size(FullCliffords)

(11520,)

In [32]:
FullCliffords[1]

4x4 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im

## Check we have done it correct (unitary 2)

In [33]:
sum = 0
numb = size(FullCliffords,1)
p=Progress(numb,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(FullCliffords[i])'*FullCliffords[j]))^4
    end
    next!(p)
end
print("$sum -> $(sum/numb^2))\n")

Progress: 100% Time: 0:05:59
2.6542079999999997e8 -> 1.9999999999999998)


##Hmm probably a rounding error.

## So recap
- we have in sl2Cliffords each of the 4 x 4 matrices that form our smaller unitary 2 design (960)
- we have in FullCLiffords each of the 4 x 4 matrices taht form our full clifford group (11520)
- we are going to make a SuperOperator (Pauli Basis) for each of the full clifford group and store this
- we are going to store and INDEX for the sl2 cliffords to the SuperOperator index
- we are going to identify our generators, superoperator and find their indices.
- we are going to make a tree with 11520 vertices
- then we multiply each $CL_i^\{dagger},CL_j$ to see if they are a generator and set the edge appropriately 

In [34]:
#Granade inspired
function to_superpauli(S):
    """
    Converts a superoperator in the column-stacking basis to the Pauli basis. Note that
    the argument is assumed to act on qubits.
    """
    sqobj=qt.to_super(S)
    nq=int(np.log2(sqobj[:shape][1])/2)
    B=pycall(qt.superop_reps["_pauli_basis"],PyAny,nq)
    sq=np.sqrt(2^nq)
    B=pyeval("B/sq",PyAny,B=B,nq=nq,sq=sq)
    B[:dims] = sqobj[:dims]
    sqobj = pycall(B[:dag]()["__mul__"],PyAny,pycall(sqobj["__mul__"],PyAny,B))
    return sqobj
end

to_superpauli (generic function with 1 method)

In [35]:
SuperCliffs = [int(real(to_superpauli(qt.Qobj(FullCliffords[i]))[:full]())) for i =1:length(FullCliffords)];

In [36]:
indexOfSl2 = [findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(sl2Cliffords[i]))[:full]()))) for i=1:length(sl2Cliffords)]

960-element Array{Any,1}:
  1937
  1939
  1945
  1947
  1938
  1940
  1946
  1948
  1941
  1943
  1949
  1951
  1942
     ⋮
 10182
 10192
 10185
 10179
 10188
 10178
 10183
 10189
 10191
 10181
 10180
 10186

In [176]:
## So our generators
# start with some easy ones (note forcing to complex for just now)
identity = [1 0im;0 1]
pauliX = [0im 1;1 0]
pauliZ = [1 0im;0 -1]
pauliY = [0 -1im;1im 0]
phaseGate = [1 0;0 1im]
rphaseGate = phaseGate*phaseGate*phaseGate
shadmard=1/sqrt(2)*[1 1+0im;1 -1]
cnot12 = [1 0 0 0im;0 1 0 0;0 0 0 1;0 0 1 0]
cnot21 = [1 0 0 0im;0 0 0 1;0 0 1 0;0 1 0 0]
#Assume that can't parallelise gates so we have
Generators=Array{Complex{Float64},2}[]
push!(Generators,[kron(identity,pauliX)])
push!(Generators,[kron(pauliX,identity)])
push!(Generators,[kron(pauliX,pauliX)])
push!(Generators,[kron(pauliY,pauliY)])
push!(Generators,[kron(pauliZ,pauliZ)])
push!(Generators,[kron(identity,pauliZ)])
push!(Generators,[kron(pauliZ,identity)])
push!(Generators,[kron(identity,pauliY)])
push!(Generators,[kron(pauliY,identity)])
push!(Generators,[kron(identity,phaseGate)])
push!(Generators,[kron(phaseGate,identity)])
push!(Generators,[kron(identity,rphaseGate)])
push!(Generators,[kron(rphaseGate,identity)])
push!(Generators,[kron(identity,shadmard)])
push!(Generators,[kron(shadmard,identity)])
push!(Generators,[kron(phaseGate,phaseGate)])
push!(Generators,[kron(rphaseGate,rphaseGate)])
push!(Generators,[kron(shadmard,shadmard)])
push!(Generators,cnot12)
push!(Generators,cnot21);

In [177]:
indexOfGens = [findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(Generators[i]))[:full]()))) for i=1:length(Generators)]

20-element Array{Any,1}:
     9
     5
    13
    16
     4
     3
     2
    11
     6
  9603
    34
  9601
    33
  1921
    17
  9636
  9633
  1937
  5825
 10561

In [47]:
#actually lets check we have a group.


Array{Complex{Int64},2}

In [40]:
rphaseGate

2x2 Array{Complex{Int64},2}:
 1+0im  0+0im
 0+0im  0-1im

In [59]:
SuperSL2 = [int(real(to_superpauli(qt.Qobj(sl2Cliffords[i]))[:full]())) for i =1:length(sl2Cliffords)];

In [62]:
length(SuperSL2)

960

In [64]:
for i=1:length(SuperSL2)
    for j=1:length(SuperSL2)
        if (findfirst(SuperSL2,SuperSL2[i]*SuperSL2[j])==0)
            print("Check ",i,", ",j)
        end
    end
end

In [178]:
GensinSL2=[findfirst(SuperSL2,int(real(to_superpauli(qt.Qobj(Generators[i]))[:full]()))) for i=1:length(Generators)]

20-element Array{Any,1}:
 450
 453
 454
 464
 459
 451
 457
 452
 461
   0
   0
   0
   0
   0
   0
 507
 497
   1
   0
   0

In [70]:
Pkg.add("Graphs")

INFO: Cloning cache of DataStructures from git://github.com/JuliaLang/DataStructures.jl.git
INFO: Cloning cache of Docile from git://github.com/MichaelHatherly/Docile.jl.git
INFO: Cloning cache of Graphs from git://github.com/JuliaLang/Graphs.jl.git
INFO: Installing DataStructures v0.3.9
INFO: Installing Docile v0.5.7
INFO: Installing Graphs v0.5.4
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Graphs
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [131]:
Adjacency=zeros(Int64,length(FullCliffords),length(FullCliffords))

11520x11520 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0 

In [179]:
SuperGenerators=[int(real(to_superpauli(qt.Qobj(Generators[i]))[:full]())) for i =1:length(Generators)];

In [180]:
p=Progress(length(SuperCliffs))
for i=1:length(SuperCliffs)
    for j=1:length(SuperCliffs)
        if (findfirst(SuperGenerators,SuperCliffs[i]'*SuperCliffs[j])!=0)
            Adjacency[i,j]=1
        end
    end
    next!(p)
end


Progress: 100%|█████████████████████████████████████████| Time: 0:28:00


In [134]:
Adjacency

11520x11520 Array{Int64,2}:
 0  1  1  0  1  1  0  0  1  0  1  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  1  1  1  0  0  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  1  0  0  1  1  1  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  1  0  0  0  1  1  0  1  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  0  1  1  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 1  1  0  0  1  0  0  1  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  1  1  0  0  1  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  1  0  1  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  0  0  0  0  1  1  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  1  0  0  1  1     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  1  0  0  0  0  0  1  0  0  1  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  1  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  1  0  1  1  0  0 

In [182]:
findfirst(SuperCliffs,SuperCliffs[18])

18

In [231]:
writecsv("adjacencyF.csv",AdjacencyF)

In [184]:
findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(kron(identity,identity)))[:full]())))

1

In [185]:
Adjacency[1,:]

1x11520 Array{Int64,2}:
 0  1  1  1  1  1  0  0  1  0  1  0  1  …  0  0  0  0  0  0  0  0  0  0  0  0

In [186]:
#So these are the ones that are reachable using one generator.

In [187]:
FullCliffordCommands[5]

8-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "phase(1)"    
 "phase(1)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "output(svec)"

In [188]:
findfirst(SuperGenerators,SuperCliffs[5])

2

In [189]:
pathLength=zeros(Int64,size(Adjacency,1));

In [190]:
iteration=1
for i=1:size(Adjacency,1)
    if (pathLength[i]==0 && Adjacency[1,i] != 0)
        pathLength[i]=iteration
    end
end

In [218]:
pathLength

11520-element Array{Int64,1}:
 2
 1
 1
 1
 1
 1
 2
 2
 1
 2
 1
 2
 1
 ⋮
 4
 4
 5
 4
 4
 4
 4
 4
 4
 4
 4
 4

# Turns out that we need it to be Float64 (well not Int8/Int64 anyway) for the multiplicaiton
# to be done in a sane time. I presume its to do with LAPACK

In [192]:
AdjacencyF=convert(Array{Float64,2},Adjacency)


11520x11520 Array{Float64,2}:
 0.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  1.0  1.0  1.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  0.0  0.0  0.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  1.0  0.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  1.0  1.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  1.0  0.0  0.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  1.0  0.0  1.0

In [193]:
Adjacency2= AdjacencyF*AdjacencyF

11520x11520 Array{Float64,2}:
 20.0   6.0   6.0   6.0   4.0   4.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  6.0  20.0   6.0   6.0   4.0   4.0      0.0   0.0   0.0   0.0   0.0   0.0
  6.0   6.0  20.0   6.0   6.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  6.0   6.0   6.0  20.0   6.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  4.0   4.0   6.0   6.0  20.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  4.0   4.0   6.0   6.0   6.0  20.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  6.0   6.0   4.0   4.0   6.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  6.0   6.0   4.0   4.0   6.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  4.0   6.0   4.0   6.0   4.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  6.0   4.0   6.0   4.0   6.0   4.0      0.0   0.0   0.0   0.0   0.0   0.0
  4.0   6.0   4.0   6.0   6.0   4.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  6.0   4.0   6.0   4.0   4.0   6.0      0.0   0.0   0.0   0.0   0.0   0.0
  4.0   6.0   6.0   4.0   4.0   6.0      0.0   0.0   0.0   0.0   0.0  

##So what we have here is showing us the number of ways we can get from one Clifford (the row) to another (the column) in two moves.

Just now I am only interested in the number of path length, not the number of paths.

In [194]:
iteration=2
for i=1:size(Adjacency2,1)
    if (pathLength[i]==0 && Adjacency2[1,i] != 0)
        pathLength[i]=iteration
    end
end

In [195]:
pathLength

11520-element Array{Int64,1}:
 2
 1
 1
 1
 1
 1
 2
 2
 1
 2
 1
 2
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [196]:
AdjacencyN = AdjacencyF * Adjacency2

11520x11520 Array{Float64,2}:
  72.0  106.0  106.0  104.0   88.0  …    0.0    0.0    0.0    0.0    0.0
 106.0   72.0  104.0  106.0   86.0       0.0    0.0    0.0    0.0    0.0
 106.0  104.0   72.0  106.0   57.0       0.0    0.0    0.0    0.0    0.0
 104.0  106.0  106.0   72.0   55.0       0.0    0.0    0.0    0.0    0.0
  88.0   86.0   57.0   55.0   72.0       0.0    0.0    0.0    0.0    0.0
  86.0   88.0   55.0   57.0  106.0  …    0.0    0.0    0.0    0.0    0.0
  57.0   55.0   88.0   86.0  106.0       0.0    0.0    0.0    0.0    0.0
  55.0   57.0   86.0   88.0  104.0       0.0    0.0    0.0    0.0    0.0
  88.0   57.0   86.0   55.0   86.0       0.0    0.0    0.0    0.0    0.0
  57.0   88.0   55.0   86.0   55.0       0.0    0.0    0.0    0.0    0.0
  86.0   55.0   88.0   57.0   55.0  …    1.0    0.0    0.0    0.0    0.0
  55.0   86.0   57.0   88.0   86.0       0.0    0.0    0.0    0.0    0.0
  86.0   55.0   55.0   86.0   88.0       0.0    0.0    1.0    0.0    0.0
   ⋮                 

In [197]:
iteration=3
for i=1:size(AdjacencyN,1)
    if (pathLength[i]==0 && AdjacencyN[1,i] != 0)
        pathLength[i]=iteration
    end
end

In [198]:
p=Progress(7,1)
for co=4:10
    AdjacencyN = AdjacencyF * AdjacencyN
    for i=1:size(AdjacencyN,1)
        if (pathLength[i]==0 && AdjacencyN[1,i] != 0)
            pathLength[i]=co
        end
    end
    next!(p)
end

Progress: 100% Time: 0:03:54


In [199]:
pathLength

11520-element Array{Int64,1}:
 2
 1
 1
 1
 1
 1
 2
 2
 1
 2
 1
 2
 1
 ⋮
 4
 4
 5
 4
 4
 4
 4
 4
 4
 4
 4
 4

In [200]:
for i=1:11520
    if pathLength[i]==0 
        print("$i is still zero\n")
    end
end

In [201]:
mean(pathLength)

4.852256944444444

In [202]:
count = 0
for i = 1:length(indexOfSl2)
    count += pathLength[indexOfSl2[i]]
end


In [203]:
count

4532

In [204]:
count = count/length(indexOfSl2)

4.720833333333333

In [205]:
maximum(pathLength)

7

In [206]:
maxL=0
for i = 1:length(indexOfSl2)
    if pathLength[indexOfSl2[i]] > maxL
        maxL =  pathLength[indexOfSl2[i]] 
    end
end

In [207]:
maxL

6

###So I should be able to do this, using just the generators in the Sl2 group, (although I suspect gate lengths will be longer?)


###first just re-check SL2 is a group, we checked that any group member* another group member = a group member, but I think I also need to check that each group member has an inverse in the group

In [209]:
Superidentity = int(real(to_superpauli(qt.Qobj(kron(identity,identity)))[:full]()))
print(findfirst(SuperSL2,Superidentity))

449

In [247]:
for i = 1:length(SuperSL2)
    done  = 0
    for j=1:length(SuperSL2)
        if SuperSL2[i]*SuperSL2[j] == Superidentity
            done=1
            continue
        end
    end
    if (done == 0) print("Failed with $i\n")
    end
end


In [211]:
SuperSL2[449]

16x16 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1

In [232]:
##Playing with the SL 60

In [235]:
sl60Cliffords= Array{Complex{Float64},2}[]
sl60CliffordCommands = Array{ASCIIString,1}[]
n=60
p=Progress(n,1)
for i=1:60
      state=setup(2)
      state[:,1:4]=stabs[i]
      decomposeState(state,true)
      push!(sl60CliffordCommands,commands)
      push!(sl60Cliffords,makeFromCommand(commands))
end

In [236]:
size(sl60Cliffords)

(60,)

In [241]:
#Find the index of these cliffords in the FullClifford group
Super60Cliffords=[int(real(to_superpauli(qt.Qobj(sl60Cliffords[i]))[:full]())) for i =1:length(sl60Cliffords)];

In [243]:
indexOfSl60 = [findfirst(SuperCliffs,Super60Cliffords[i]) for i=1:length(Super60Cliffords)]

60-element Array{Any,1}:
  1937
  2897
  7937
  8897
   113
  6113
  6593
   593
  1585
  4465
  9985
  7105
  3649
     ⋮
 10865
  1185
  7393
  4657
  3665
  8385
 10273
  1777
  4865
  6705
  1393
 10177

In [246]:
for i = 1:length(Super60Cliffords)
    done  = 0
    for j=1:length(Super60Cliffords)
        if Super60Cliffords[i]*Super60Cliffords[j] == Superidentity
            done=1
            continue
        end
    end
    if (done == 0) print("Failed with $i\n")
    end
end


Failed with 7
Failed with 11
Failed with 14
Failed with 15
Failed with 18
Failed with 19
Failed with 21
Failed with 22
Failed with 24
Failed with 25
Failed with 28
Failed with 30
Failed with 31
Failed with 32
Failed with 33
Failed with 34
Failed with 35
Failed with 37
Failed with 39
Failed with 40
Failed with 41
Failed with 42
Failed with 44
Failed with 46
Failed with 47
Failed with 50
Failed with 51
Failed with 53
Failed with 56
Failed with 57
Failed with 59
Failed with 60


##So this means that the 60 SL group (prior to conjugation with a Pauli don't have their own inverses i.e. they do not form a group

In [250]:
CTable=zeros(60,60)
for i=1:60
    for j=1:60
        CTable[i,j]=findfirst(Super60Cliffords,Super60Cliffords[i]*Super60Cliffords[j])
    end
end

In [251]:
CTable


60x60 Array{Float64,2}:
 29.0  31.0  32.0  30.0  45.0  47.0  …  53.0  54.0  23.0  21.0  22.0  24.0
  0.0  43.0  44.0   0.0   0.0  51.0      0.0  58.0   0.0   0.0   0.0  16.0
  0.0   0.0  36.0   0.0   0.0  55.0     45.0   0.0   0.0  17.0   0.0   0.0
  0.0   0.0  40.0  38.0   0.0  59.0     49.0   0.0   0.0  25.0  26.0   0.0
 13.0   0.0  15.0   0.0  29.0   0.0      0.0   0.0  58.0  57.0   0.0   0.0
  0.0   0.0  27.0   0.0   0.0   0.0  …  37.0   0.0   0.0  49.0   0.0   0.0
  0.0   0.0  19.0   0.0   0.0   0.0     33.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  23.0   0.0   0.0   0.0     29.0   0.0   0.0   0.0   0.0   0.0
 45.0   0.0  46.0   0.0  13.0   0.0      0.0   0.0  36.0  33.0   0.0   0.0
  0.0   0.0  58.0   0.0   0.0   0.0     13.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  50.0   0.0   0.0   0.0  …  25.0   0.0   0.0  37.0   0.0   0.0
  0.0   0.0  54.0   0.0   0.0   0.0      0.0   0.0   0.0  29.0   0.0   0.0
  5.0   0.0   8.0   0.0   9.0   0.0      0.0   0.0  33.0  36.0   0.0   0.0
 

In [260]:
for i=1:60
    if (sort(vec(int(CTable[i,:])))) == [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
        print(i,"\n")
    end
end


1
29


In [263]:
Super60Cliffords[29]

16x16 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1

In [268]:
sl60CliffordCommands[1]

4-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "output(svec)"

In [269]:
stabs[1]

4x4 Array{Int64,2}:
 0  0  1  0
 0  0  0  1
 1  0  0  0
 0  1  0  0

In [270]:
stabs[3]

4x4 Array{Int64,2}:
 0  0  1  0
 0  0  0  1
 1  0  1  0
 0  1  0  1

In [271]:
stabs[36]

4x4 Array{Int64,2}:
 1  0  1  0
 0  1  0  1
 1  0  0  0
 0  1  0  0

In [275]:
findfirst(SuperCliffs,Super60Cliffords[36])

4113

In [278]:
pathLength[4113]

2